In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_469252/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
7,model_14_5_14,0.793455,0.424424,0.660801,0.567801,24.909344,81.539421,57.884003,70.407455,"Hidden Size=[70], regularizer=0.1, learning_ra..."
8,model_14_5_13,0.793333,0.425137,0.661735,0.568589,24.924143,81.438347,57.724739,70.278999,"Hidden Size=[70], regularizer=0.1, learning_ra..."
9,model_14_5_15,0.792686,0.420828,0.659611,0.565559,25.002161,82.048798,58.087082,70.772690,"Hidden Size=[70], regularizer=0.1, learning_ra..."
10,model_14_5_16,0.792685,0.412508,0.656281,0.560086,25.002230,83.227554,58.655453,71.664215,"Hidden Size=[70], regularizer=0.1, learning_ra..."
11,model_14_5_11,0.792604,0.432797,0.670031,0.576205,25.012068,80.353249,56.309074,69.038353,"Hidden Size=[70], regularizer=0.1, learning_ra..."
12,model_14_5_12,0.792475,0.425048,0.661085,0.568228,25.027588,81.450989,57.835545,70.337852,"Hidden Size=[70], regularizer=0.1, learning_ra..."
13,model_14_5_17,0.791733,0.408553,0.653066,0.556680,25.117075,83.787834,59.204136,72.219040,"Hidden Size=[70], regularizer=0.1, learning_ra..."
14,model_14_5_10,0.790978,0.434956,0.674780,0.579541,25.208160,80.047447,55.498512,68.495003,"Hidden Size=[70], regularizer=0.1, learning_ra..."
15,model_14_5_18,0.790663,0.397027,0.646997,0.548383,25.246164,85.420547,60.239815,73.570793,"Hidden Size=[70], regularizer=0.1, learning_ra..."
16,model_14_5_19,0.790347,0.394416,0.646723,0.547046,25.284161,85.790497,60.286484,73.788612,"Hidden Size=[70], regularizer=0.1, learning_ra..."
